### Análise Exploratória dos Dados (EDA)

Este notebook tem como objetivo explorar estatisticamente e visualmente o conjunto de dados de imóveis do Distrito Federal, previamente tratado e padronizado.


In [5]:
# Bibliotecas

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv(r'C:\analise_imoveis\data\processed\imoveis_df_limpo.csv')

df.head()

,BAIRRO,VALOR,AREA,QUARTOS
0,ASA NORTE,1400000.0,141.0,2.0
1,SUDOESTE,1529000.0,79.0,2.0
2,NOROESTE,1395000.0,85.0,2.0
3,AGUAS CLARAS,450000.0,58.0,2.0
4,SUDOESTE,3580000.0,186.0,4.0


In [7]:
# Copia do dataset
df_eda = df.copy()

#### 1. Visão geral do conjunto de dados

Nesta etapa inicial da análise exploratória, busca-se compreender a estrutura do dataset após o processo de limpeza e padronização. São avaliados o número de registros, tipos das variáveis e estatísticas descritivas básicas, a fim de verificar a consistência dos dados antes das análises visuais e estatísticas mais aprofundadas.


In [8]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15048 entries, 0 to 15047
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAIRRO   14741 non-null  object 
 1   VALOR    15048 non-null  float64
 2   AREA     15048 non-null  float64
 3   QUARTOS  15048 non-null  float64
dtypes: float64(3), object(1)
memory usage: 470.4+ KB


##### Valores ausentes na variável BAIRRO

Observa-se que a variável **BAIRRO** apresenta um número inferior de registros não nulos em relação ao total do dataset. Isso indica a presença de anúncios cujo texto não permitiu a identificação clara da região administrativa, mesmo após o processo de padronização.

Esses registros permanecem no conjunto de dados por ainda conterem informações válidas de preço, área e quantidade de quartos, podendo ser avaliados posteriormente quanto à sua relevância para análises por região.


In [9]:
df.describe()

,VALOR,AREA,QUARTOS
count,1.504800e+04,15048.000000,15048.000000
mean,2.142880e+06,326.432948,3.353668
std,3.687005e+07,3845.427201,2.655289
min,1.700000e+04,0.000000,1.000000
25%,5.500000e+05,72.000000,2.000000
50%,9.950000e+05,164.000000,3.000000
75%,1.789000e+06,340.000000,4.000000
max,3.800000e+09,317200.000000,100.000000


##### Definição do escopo analítico

Durante a análise inicial, foi identificado que o conjunto de dados contém valores extremos
(outliers) em múltiplas variáveis, especialmente em preço, área e número de quartos.

Esses valores incluem imóveis que fogem do perfil residencial padrão, como:
- terrenos e fazendas,
- prédios inteiros,
- imóveis comerciais,
- anúncios com valores inconsistentes.

Como o objetivo desta análise é compreender o comportamento do mercado imobiliário residencial
típico do Distrito Federal, optou-se por aplicar filtros que delimitam um escopo mais condizente
com a realidade observada para esse tipo de imóvel.


In [10]:
df_filter = df_eda[
    (df_eda['VALOR'] <= 3_500_000) &
    (df_eda['QUARTOS'] <= 4) &
    (df_eda['AREA'] >= 60) &
    (df_eda['AREA'] <= 1000)
].copy()

In [11]:
print(f'Tamanho orginal: {df_eda.shape[0]}')
print(f'Tamanho após filtro: {df_filter.shape[0]}')

Tamanho orginal: 15048
Tamanho após filtro: 9836


- Após a aplicação dos filtros, o conjunto de dados foi reduzido, removendo observações
que fogem do escopo residencial padrão. Essa etapa melhora a interpretabilidade das
análises estatísticas e visuais subsequentes.


##### Critérios de filtragem aplicados

Os seguintes filtros foram definidos com base em conhecimento prévio do dataset e
em limites plausíveis para imóveis residenciais urbanos:

- **Preço máximo**: R$ 3.500.000  
  → Valores acima desse limite representam uma parcela muito específica e rara do mercado,
    além de distorcerem medidas estatísticas como média e desvio padrão.

- **Número máximo de quartos**: 4  
  → Imóveis com número superior de quartos tendem a representar construções atípicas
    ou anúncios inconsistentes para o escopo proposto.

- **Área entre 60 m² e 1.000 m²**  
  → O limite inferior remove anúncios incompatíveis com residências completas, enquanto
    o limite superior evita a inclusão de terrenos, chácaras ou grandes propriedades.


In [12]:
df_filter.describe()

,VALOR,AREA,QUARTOS
count,9.836000e+03,9836.000000,9836.000000
mean,1.229527e+06,213.476413,3.130439
std,7.341647e+05,152.669548,0.749149
min,4.000000e+04,60.000000,1.000000
25%,6.600000e+05,97.000000,3.000000
50%,1.080000e+06,166.000000,3.000000
75%,1.600000e+06,294.250000,4.000000
max,3.500000e+06,1000.000000,4.000000


> Importante: os filtros aplicados não têm como objetivo eliminar outliers estatísticos
> de forma automática, mas sim delimitar o escopo da análise a um segmento específico
> do mercado imobiliário. Análises sobre imóveis de alto padrão ou propriedades atípicas
> podem ser realizadas em estudos separados.

In [13]:
df_filter.isna().sum()

BAIRRO     162
VALOR        0
AREA         0
QUARTOS      0
dtype: int64

In [14]:
df_filter.describe(include=object)

,BAIRRO
count,9674
unique,37
top,AGUAS CLARAS
freq,1944


In [15]:
# Maior valor
df_filter.loc[[df_filter['VALOR'].idxmax()]]

,BAIRRO,VALOR,AREA,QUARTOS
454,JARDIM BOTANICO,3500000.0,480.0,4.0


In [16]:
# Menor valor
df_filter.loc[[df_filter['VALOR'].idxmin()]]

,BAIRRO,VALOR,AREA,QUARTOS
11463,NaN,40000.0,100.0,2.0


In [17]:
# Maior Área
df_filter.loc[[df_filter['AREA'].idxmax()]]

,BAIRRO,VALOR,AREA,QUARTOS
1615,SOBRADINHO,2190000.0,1000.0,4.0


In [18]:
# Menor Área
df_filter.loc[[df_filter['AREA'].idxmin()]]

,BAIRRO,VALOR,AREA,QUARTOS
29,AGUAS CLARAS,365000.0,60.0,2.0


In [19]:
# Maior valor por bairro
df_filter.loc[df_filter.groupby('BAIRRO')['VALOR'].idxmax()]

,BAIRRO,VALOR,AREA,QUARTOS
3193,AGUAS CLARAS,3500000.0,260.0,4.0
3863,ARNIQUEIRA,3500000.0,440.0,4.0
1661,ASA NORTE,3500000.0,257.0,4.0
40,ASA SUL,3200000.0,211.0,4.0
11680,BRAZLANDIA,849000.0,368.0,3.0
10318,CANDANGOLANDIA,1900000.0,600.0,3.0
14289,CEILANDIA,2150000.0,580.0,2.0
7934,CRUZEIRO,3200000.0,360.0,4.0
12610,FERCAL,2750000.0,92.0,3.0
3343,GAMA,3450000.0,600.0,2.0


In [20]:
# Menor valor por bairro
df_filter.loc[df_filter.groupby('BAIRRO')['VALOR'].idxmin()]

,BAIRRO,VALOR,AREA,QUARTOS
6772,AGUAS CLARAS,150000.0,62.0,2.0
12923,ARNIQUEIRA,125000.0,200.0,4.0
10376,ASA NORTE,250000.0,60.0,1.0
6990,ASA SUL,360000.0,78.0,1.0
1119,BRAZLANDIA,320000.0,130.0,3.0
11309,CANDANGOLANDIA,350000.0,128.0,3.0
15003,CEILANDIA,175000.0,75.0,3.0
12856,CRUZEIRO,395000.0,300.0,3.0
12610,FERCAL,2750000.0,92.0,3.0
6713,GAMA,95000.0,400.0,4.0


In [21]:
df_filter['BAIRRO'].value_counts()

BAIRRO
AGUAS CLARAS          1944
JARDIM BOTANICO       1006
SOBRADINHO             880
VICENTE PIRES          697
GUARA                  560
TAGUATINGA             523
ASA NORTE              490
NOROESTE               454
SAMAMBAIA              445
ARNIQUEIRA             426
ASA SUL                373
SUDOESTE               257
PARK WAY               211
CEILANDIA              205
GAMA                   198
LAGO NORTE             197
PARK SUL               160
RIACHO FUNDO           131
LAGO SUL               100
CRUZEIRO                74
JARDINS MANGUEIRAL      65
NUCLEO BANDEIRANTE      50
OCTOGONAL               37
TAQUARI                 35
SANTA MARIA             29
RECANTO DAS EMAS        27
CANDANGOLANDIA          23
PARANOA                 22
PLANALTINA              20
SAO SEBASTIAO           14
ITAPOA                   6
SOL NASCENTE             6
BRAZLANDIA               3
VARJAO                   2
POR DO SOL               2
SIA                      1
FERCAL               

In [22]:
df_filter['BAIRRO'].value_counts()

BAIRRO
AGUAS CLARAS          1944
JARDIM BOTANICO       1006
SOBRADINHO             880
VICENTE PIRES          697
GUARA                  560
TAGUATINGA             523
ASA NORTE              490
NOROESTE               454
SAMAMBAIA              445
ARNIQUEIRA             426
ASA SUL                373
SUDOESTE               257
PARK WAY               211
CEILANDIA              205
GAMA                   198
LAGO NORTE             197
PARK SUL               160
RIACHO FUNDO           131
LAGO SUL               100
CRUZEIRO                74
JARDINS MANGUEIRAL      65
NUCLEO BANDEIRANTE      50
OCTOGONAL               37
TAQUARI                 35
SANTA MARIA             29
RECANTO DAS EMAS        27
CANDANGOLANDIA          23
PARANOA                 22
PLANALTINA              20
SAO SEBASTIAO           14
ITAPOA                   6
SOL NASCENTE             6
BRAZLANDIA               3
VARJAO                   2
POR DO SOL               2
SIA                      1
FERCAL               

In [23]:
# Bairros com menos de 100 ocorrencias
df_filter['BAIRRO'].value_counts().loc[lambda x: x < 100].sum()

np.int64(417)

#### Filtragem do Conjunto de Dados para Análises Regionais e Modelagem

Durante a análise exploratória, identificou-se a presença de dois fatores que poderiam comprometer a robustez estatística das análises por bairro e a construção de um modelo preditivo confiável:

- Registros sem identificação de bairro (valores ausentes na variável **BAIRRO**);
- Bairros com baixo volume de observações, o que torna métricas como média, mediana e dispersão estatisticamente instáveis.

Ao todo, esses casos representam uma parcela reduzida do conjunto de dados, mas com potencial elevado de introduzir ruído e vieses nas análises comparativas e no modelo de machine learning.

Dessa forma, optou-se por criar um novo conjunto de dados filtrado, removendo:
- Registros com **BAIRRO** ausente;
- Bairros com menos de **100 observações**.

Essa decisão visa garantir maior consistência estatística, melhorar a capacidade de generalização do modelo e proporcionar uma experiência mais confiável em uma futura aplicação web interativa.



In [24]:
# Contagem de registros por bairros
contagem_de_bairros = df_filter['BAIRRO'].value_counts()

# Bairros com com pelo menos 100 observações
bairros_validos = contagem_de_bairros[contagem_de_bairros >= 100].index

# Criando o novo dataset filtrado
df_modelo = df_filter[
    df_filter['BAIRRO'].notna() &
    df_filter['BAIRRO'].isin(bairros_validos)
].copy()

In [25]:
# Verificação
df_modelo['BAIRRO'].value_counts()

BAIRRO
AGUAS CLARAS       1944
JARDIM BOTANICO    1006
SOBRADINHO          880
VICENTE PIRES       697
GUARA               560
TAGUATINGA          523
ASA NORTE           490
NOROESTE            454
SAMAMBAIA           445
ARNIQUEIRA          426
ASA SUL             373
SUDOESTE            257
PARK WAY            211
CEILANDIA           205
GAMA                198
LAGO NORTE          197
PARK SUL            160
RIACHO FUNDO        131
LAGO SUL            100
Name: count, dtype: int64

In [26]:
df_modelo.isna().sum()

BAIRRO     0
VALOR      0
AREA       0
QUARTOS    0
dtype: int64

In [27]:
print('Dataset orginal:', df.shape)
print('Dataset da primeira analise:', df_filter.shape)
print('Dataset da ultima analise:', df_modelo.shape)

Dataset orginal: (15048, 4)
Dataset da primeira analise: (9836, 4)
Dataset da ultima analise: (9257, 4)


In [28]:
# Vamos realizar um novo descibre
df_modelo.describe().round(2)

,VALOR,AREA,QUARTOS
count,9257.00,9257.00,9257.00
mean,1249305.41,215.48,3.14
std,729950.39,152.94,0.75
min,75000.00,60.00,1.00
25%,690000.00,99.00,3.00
50%,1100000.00,168.00,3.00
75%,1650000.00,300.00,4.00
max,3500000.00,1000.00,4.00


#### Distribuição das Variáveis Numéricas

Antes de aprofundar a análise por região, é fundamental compreender o
comportamento geral das principais variáveis numéricas do conjunto de dados,
avaliando assimetrias, dispersões e possíveis padrões globais.


In [29]:
import plotly.express as px

- _Todos os gráficos utilizam uma paleta fixa inspirada na identidade visual de Brasília, garantindo consistência visual e melhor experiência para o usuário._

In [30]:
PALETA_BRASILIA = {
    "azul": "#1F4FD8",      
    "verde": "#1B5E20",     
    "cinza": "#4F4F4F",     
    "cinza_claro": "#BDBDBD",
    "amarelo": "#F2C94C"    
}


In [31]:
# Histograma
fig = px.histogram(
    df_modelo,
    x='VALOR',
    nbins=40,
    title= 'Distribuição dos Preços dos imóveis em Brasilia',
    labels = {'VALOR': 'Preço do imóvel (R$)'},
    color_discrete_sequence=[PALETA_BRASILIA['azul']]
)

fig.update_layout(
    xaxis_tickformat=',.0f',
    bargap=0.05,
    plot_bgcolor='white'
)

fig.show()

In [32]:
# Boxplot

fig = px.box(
    df_modelo,
    y='VALOR',
    title='Distribuição dos Preços dos imóveis - BoxPlot',
    labels={'VALOR': 'Preço do Imóvel R$'},
    color_discrete_sequence=[PALETA_BRASILIA['verde']]
)

fig.update_layout(
    yaxis_tickformat=',0f',
    plot_bgcolor='white'
)

fig.show()

- _Os gráficos indicam uma distribuição assimétrica à direita, com presença de outliers superiores plausíveis. Observa-se ainda um comportamento bimodal no histograma de preços, sugerindo a existência de diferentes segmentos no mercado imobiliário do Distrito Federal, com concentrações em faixas de menor e médio-alto valor._

#### Criação da variável VALOR por m²

Para tornar a análise de preços mais comparável entre imóveis de diferentes tamanhos, foi criada a variável **VALOR por m²**, obtida pela razão entre o valor total do imóvel e sua área construída.

Essa métrica permite reduzir o efeito do tamanho do imóvel sobre o preço absoluto, facilitando comparações entre bairros e a identificação de regiões mais valorizadas ou subvalorizadas.


In [33]:
# Coferencia basica
df_modelo[['VALOR', 'AREA']].describe().round(2)

,VALOR,AREA
count,9257.00,9257.00
mean,1249305.41,215.48
std,729950.39,152.94
min,75000.00,60.00
25%,690000.00,99.00
50%,1100000.00,168.00
75%,1650000.00,300.00
max,3500000.00,1000.00


In [34]:
# Criando o valor por m²
df_modelo['VALOR_M2'] = df_modelo['VALOR'] / df_modelo['AREA']

In [35]:
# Conferindo se deu tudo certo
df_modelo.head(20).round(2)

,BAIRRO,VALOR,AREA,QUARTOS,VALOR_M2
0,ASA NORTE,1400000.0,141.0,2.0,9929.08
1,SUDOESTE,1529000.0,79.0,2.0,19354.43
2,NOROESTE,1395000.0,85.0,2.0,16411.76
5,AGUAS CLARAS,850000.0,90.0,3.0,9444.44
6,GUARA,850000.0,68.0,2.0,12500.00
7,ASA NORTE,1500000.0,74.0,2.0,20270.27
8,ASA NORTE,1390000.0,73.0,2.0,19041.10
9,LAGO NORTE,2290000.0,411.0,4.0,5571.78
11,ASA NORTE,1430000.0,100.0,3.0,14300.00
12,JARDIM BOTANICO,1400000.0,200.0,3.0,7000.00


In [36]:
df_modelo.describe().round(2)

,VALOR,AREA,QUARTOS,VALOR_M2
count,9257.00,9257.00,9257.00,9257.00
mean,1249305.41,215.48,3.14,7287.90
std,729950.39,152.94,0.75,4340.24
min,75000.00,60.00,1.00,237.50
25%,690000.00,99.00,3.00,3888.89
50%,1100000.00,168.00,3.00,6354.92
75%,1650000.00,300.00,4.00,9583.33
max,3500000.00,1000.00,4.00,39743.59


In [37]:
df_modelo['VALOR_M2'].quantile([0.01, 0.05, 0.95, 0.99]).round(2)

0.01     1250.00
0.05     2063.93
0.95    16000.00
0.99    19021.74
Name: VALOR_M2, dtype: float64

- _Antes de entrar nos bairros, precisamos entender como o valor do m² se comporta no DF como um todo._

In [38]:
fig = px.histogram(
    df_modelo,
    x='VALOR_M2',
    nbins=40,
    title= 'Distribuição do valor do m² no Distrito Federal',
    labels = {'VALOR_m²': 'Preço do imóvel por m² (R$)'},
    color_discrete_sequence=[PALETA_BRASILIA['azul']]
)

fig.update_layout(
    xaxis_tickformat=',.0f',
    bargap=0.05,
    plot_bgcolor='white'
)

fig.show()

In [39]:
# Boxplot

fig = px.box(
    df_modelo,
    y='VALOR_M2',
    title='Distribuição do valor do m² no Distrito Federal - BoxPlot',
    labels={'VALOR_M2': 'Preço do imóvel por m² (R$)'},
    color_discrete_sequence=[PALETA_BRASILIA['verde']]
)

fig.update_layout(
    yaxis_tickformat=',2f',
    plot_bgcolor='white'
)

fig.show()

_Os gráficos de **histograma** e **boxplot** indicam que o valor do metro quadrado apresenta uma distribuição assimétrica à direita, com forte concentração de imóveis nas faixas mais baixas de preço e uma cauda estendida para valores mais elevados._

_A maior densidade de observações está entre **R$ 3.000** e **R$ 4.000** por `m²`, enquanto a mediana se encontra em aproximadamente **R$ 6.354/m²**, evidenciando que poucos imóveis de alto padrão elevam os valores máximos sem representar a maioria do mercado. O boxplot reforça essa assimetria ao apresentar um bigode superior mais longo e a presença de outliers superiores, que, apesar de estatisticamente extremos, são compatíveis com imóveis localizados em regiões valorizadas ou de padrão elevado._

_Esse comportamento reflete a heterogeneidade do mercado imobiliário do Distrito Federal e justifica análises segmentadas por região administrativa._

#### Valor do m² por Bairro

O mercado imobiliário apresenta forte variação regional. Por isso, analisamos o valor do metro quadrado entre os bairros com maior representatividade no conjunto de dados.

Os gráficos a seguir destacam diferenças claras entre regiões, além de mostrar a dispersão dos preços dentro de cada bairro. Essas informações ajudam a entender padrões locais e fornecem insumos importantes para a construção de modelos mais robustos.

In [40]:
# Buscando os 10 bairros mais relevantes
top_bairros = (
    df_modelo['BAIRRO']
    .value_counts()
    .head(10)
    .index
)

df_top_bairros = df_modelo[df_modelo['BAIRRO'].isin(top_bairros)]

In [41]:
# Boxplot dos bairros que aparecem com mais frquencia (Volume de anuncios)
fig = px.box(
    df_top_bairros,
    x='BAIRRO',
    y='VALOR_M2',
    title= 'Distribuição do valor do m² por bairro (top 10)',
    labels={
        'Bairro': 'Bairro',
        'VALOR_M2': 'Valor do m² (R$)'
    },
    color='BAIRRO',
    color_discrete_sequence=list(PALETA_BRASILIA.values())
)

fig.update_layout(
    yaxis_tickformat=',.2f',
    plot_bgcolor='white',
    showlegend=False
)

fig.show()

In [42]:
# Ranking por mediana
valor_m2_bairro = (
    df_modelo
    .groupby('BAIRRO')['VALOR_M2']
    .median()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

In [43]:
# Grafico de barras por mediana
fig = px.bar(
    valor_m2_bairro,
    x='VALOR_M2',
    y='BAIRRO',
    text_auto='.2s',
    title='Top 10 Bairros por Mediana do Valor do m²',
    labels={
        'BAIRRO': 'Bairro',
        'VALOR_M2': 'Mediana do Valor do m² (R$)'
    },
    color_discrete_sequence=[PALETA_BRASILIA['azul']]
)

fig.update_layout(
    yaxis_tickformat=',.2f',
    plot_bgcolor='white'
)
fig.update_yaxes(autorange="reversed")
fig.show()


In [44]:
# Tratamento, do outlier de SOBRADINHO

limite_superior = df_modelo['VALOR_M2'].quantile(0.99)

df_modelo = df_modelo[df_modelo['VALOR_M2'] <= limite_superior]

- _Observou-se a presença de outliers estatísticos em alguns bairros. Em regiões de alto padrão, esses valores refletem imóveis premium e foram mantidos. No entanto, valores extremamente discrepantes em bairros cujo padrão de mercado é inferior foram tratados para evitar distorções na análise e na modelagem._

#### Análise de correlação entre variáveis

Após o tratamento dos valores atípicos, a próxima etapa consiste em avaliar a relação entre as variáveis numéricas do conjunto de dados. A análise de correlação permite identificar associações lineares relevantes, auxiliando na compreensão de quais fatores exercem maior influência sobre o valor dos imóveis.

Além da matriz de correlação, serão utilizados gráficos de dispersão `(scatterplots)` para visualizar essas relações de forma mais intuitiva, possibilitando a identificação de padrões, tendências e possíveis heterogeneidades entre os dados.


In [45]:
# Matriz de correlação (Apenas variaveis numericas)

cols_corr = ['VALOR','AREA', 'QUARTOS', 'VALOR_M2']
corr = df_modelo[cols_corr].corr()

fig = px.imshow(
    corr,
    text_auto='.2f',
    color_continuous_scale=[
        PALETA_BRASILIA['azul'],
        PALETA_BRASILIA['verde'],
        PALETA_BRASILIA['amarelo']
    ],
    title='Matriz de Correlação entre Variáveis'
)

fig.update_layout(
    plot_bgcolor='white'
)

fig.show()

- **Matriz de correlação:** _Relações lineares entre variáveis numéricas — destaque VALOR ↔ AREA._

In [46]:
fig = px.scatter(
    df_modelo,
    x='AREA',
    y='VALOR',
    title='Relação entre Área e Valor do Imóvel',
    labels={
        'AREA': 'Área do imóvel',
        'VALOR': 'Valor do imóvel (R$)'
    },
    opacity=0.6,
    color_discrete_sequence=[PALETA_BRASILIA['verde']]
)

fig.update_layout(
    yaxis_tickformat=',.2f',
    xaxis_tickformat=',.0f',
    xaxis=dict(dtick=50),
    plot_bgcolor='white'
)

fig.show()


- **Área vs Valor:** _Preço cresce com a área, com outliers de alto padrão._

In [47]:
fig = px.scatter(
    df_modelo,
    x='AREA',
    y='VALOR_M2',
    title='Relação entre Área e Valor do m²',
    labels={
        'AREA': 'Área do imóvel (m²)',
        'VALOR_M2': 'Valor do m² (R$)'
    },
    opacity=0.6,
    color_discrete_sequence=[PALETA_BRASILIA['azul']]
)

fig.update_layout(
    yaxis_tickformat=',.2f',
    plot_bgcolor='white'
)

fig.show()


- **Área vs Valor por m²:** _Áreas maiores tendem a ter menor valor por m² (desconto de escala)._

In [48]:
fig = px.scatter(
    df_modelo,
    x='QUARTOS',
    y='VALOR_M2',
    title='Relação entre Número de Quartos e Valor do m²',
    labels={
        'QUARTO': 'Numero de quartos',
        'VALOR_M2': 'Valor do m² (R$)'
    },
    opacity=0.6,
    color_discrete_sequence=[PALETA_BRASILIA['amarelo']]
)

fig.update_layout(
    yaxis_tickformat=',.0f',
    plot_bgcolor='white'
)

fig.show()


**Quartos vs Valor por m²:** _Pouca relação clara; alta dispersão por número de quartos._

#### Transformação Logarítmica do Valor por m²

A análise exploratória revelou que o valor do metro quadrado apresenta assimetria à direita e presença de valores extremos, o que pode dificultar tanto a interpretação visual quanto o desempenho de modelos preditivos.

Para lidar com esse comportamento, foi aplicada uma transformação logarítmica na variável VALOR_M2, com o objetivo de:

- Reduzir a influência de outliers

- Suavizar a dispersão dos dados

- Tornar a relação entre as variáveis mais próxima da linearidade

- Melhorar a estabilidade e a capacidade de generalização de modelos futuros

Nos próximos gráficos e análises, será feita uma comparação entre os dados antes e depois da transformação, avaliando seus impactos na distribuição, correlação e dispersão em relação à área do imóvel.

In [49]:
# Criando a variavel log do valor do m²

df_modelo['LOG_VALOR_M2'] = np.log1p(df_modelo['VALOR_M2'])

# Vizualização da transformação
fig = px.histogram(
    df_modelo,
    x='LOG_VALOR_M2',
    nbins=40,
    title= 'Distribuição do log do valor do m² no Distrito Federal',
    labels = {'LOG_VALOR_M2': 'Log do Preço do imóvel por m² (R$)'},
    color_discrete_sequence=[PALETA_BRASILIA['azul']]
)

fig.update_layout(
    xaxis_tickformat=',.2f',
    bargap=0.05,
    plot_bgcolor='white'
)

fig.show()

# Antes e depois da transformação
print('Antes (VALOR_M2):')
print(f' Média: R$ {df_modelo["VALOR_M2"].mean():,.2f}')
print(f' Mediana: R$ {df_modelo["VALOR_M2"].median():,.2f}')
print(f' Skewness: {df_modelo["VALOR_M2"].skew():,.3f}')

print('\nDepois (LOG_VALOR_M2):')
print(f' Média Log: {df_modelo["LOG_VALOR_M2"].mean():,.3f}')
print(f' Mediana Log: {df_modelo['LOG_VALOR_M2'].median():,.3f}')
print(f' Skewness Log: {df_modelo["LOG_VALOR_M2"].skew():,.3f}')

Antes (VALOR_M2):
 Média: R$ 7,149.15
 Mediana: R$ 6,285.71
 Skewness: 0.814

Depois (LOG_VALOR_M2):
 Média Log: 8.697
 Mediana Log: 8.746
 Skewness Log: -0.402


_Testes de Sensibilidade_

In [55]:
# Correlação ANTES do log
print('=== CORRELAÇÃO ORIGINAL===')
cols_corr_original = ['VALOR','AREA', 'QUARTOS', 'VALOR_M2']
corr_original = df_modelo[cols_corr_original].corr()
print(corr_original.round(2))

# Correlação DEPOIS do log
print('\n=== CORRELAÇÃO COM LOG ===')
cols_corr_log = ['VALOR','AREA', 'QUARTOS', 'LOG_VALOR_M2']
corr_log = df_modelo[cols_corr_log].corr()
print(corr_log.round(2))

# Teste: Dispersão por faixa de área
print('\n  === DISPERSÃO POR FAIXA DE AREA ==='  )
faixas = pd.cut(
    df_modelo['AREA'],
    bins=[0, 50, 100, 150, 200, 300, 500, 1000]
)

dispersao = df_modelo.groupby(faixas)[['VALOR_M2', 'LOG_VALOR_M2']].agg(['std', 'mean'])

print(dispersao.round(2))

=== CORRELAÇÃO ORIGINAL===
          VALOR  AREA  QUARTOS  VALOR_M2
VALOR      1.00  0.45     0.44      0.40
AREA       0.45  1.00     0.47     -0.49
QUARTOS    0.44  0.47     1.00     -0.17
VALOR_M2   0.40 -0.49    -0.17      1.00

=== CORRELAÇÃO COM LOG ===
              VALOR  AREA  QUARTOS  LOG_VALOR_M2
VALOR          1.00  0.45     0.44          0.39
AREA           0.45  1.00     0.47         -0.55
QUARTOS        0.44  0.47     1.00         -0.15
LOG_VALOR_M2   0.39 -0.55    -0.15          1.00

  === DISPERSÃO POR FAIXA DE AREA ===
            VALOR_M2          LOG_VALOR_M2      
                 std     mean          std  mean
AREA                                            
(0, 50]          NaN      NaN          NaN   NaN
(50, 100]    3684.96  9099.86         0.43  9.03
(100, 150]   4388.02  8934.80         0.61  8.94
(150, 200]   4492.33  7880.44         0.62  8.79
(200, 300]   3334.41  5868.51         0.54  8.53
(300, 500]   1714.94  4210.04         0.46  8.25
(500, 1000]  14

C:\Users\mateu\AppData\Local\Temp\ipykernel_1336\799897600.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



_Vizualização: log vs Original_

In [54]:
fig = px.scatter(
    df_modelo,
    x='AREA',
    y='LOG_VALOR_M2',
    title='Relação entre Área e Log do Valor do m²',
    labels={
        'AREA': 'Área do imóvel (m²)',
        'LOG_VALOR_M2': 'Log do Valor do m² (R$)'
    },
    opacity=0.6,
    color_discrete_sequence=[PALETA_BRASILIA['verde']]
)
fig.update_layout(
    plot_bgcolor='white'
)

fig.show()

#### Conclusões da Análise Exploratória (EDA)

A análise exploratória dos dados revelou padrões consistentes entre as variáveis relacionadas aos imóveis, em especial entre a área construída e o valor do metro quadrado.

Observou-se que o **valor do m² tende a diminuir à medida que a área do imóvel aumenta**, indicando um efeito de economia de escala. Imóveis maiores, apesar de apresentarem preços totais mais elevados, costumam possuir um custo proporcionalmente menor por metro quadrado.

Inicialmente, a distribuição de `VALOR_M2` apresentou **assimetria positiva moderada**, com presença de valores extremos elevados, o que impactava a média e dificultava a identificação de relações lineares. Para mitigar esse efeito, foi aplicada uma **transformação logarítmica** na variável, resultando em uma distribuição mais simétrica e com variância mais estável entre as diferentes faixas de área.

Após a transformação, verificou-se um **fortalecimento da correlação negativa entre área e valor do m²**, além de uma redução significativa da dispersão dos dados, indicando que a relação entre essas variáveis é de natureza **multiplicativa**, e não estritamente linear.

Dessa forma, a variável `LOG_VALOR_M2` mostrou-se mais adequada como variável alvo para a etapa de modelagem, contribuindo para maior estabilidade estatística, melhor comportamento dos resíduos e maior interpretabilidade dos modelos preditivos.

Com base nessas conclusões, a próxima etapa do projeto consistirá na construção e avaliação de modelos de machine learning utilizando os dados transformados.

In [56]:
df_modelo.isnull().sum()

BAIRRO          0
VALOR           0
AREA            0
QUARTOS         0
VALOR_M2        0
LOG_VALOR_M2    0
dtype: int64

In [57]:
df_modelo.to_csv(r'C:\analise_imoveis\data\processed\imoveis_df_eda.csv', index=False)